<a href="https://colab.research.google.com/github/shashank3110/Master_Thesis_BA_DeepVis/blob/master/colab_notebooks/ba_estimation_network_saliency_maps_gcam_gcam%2B%2B_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/LLNL/fastcam.git

!pip install pytorch_gradcam   #hooks=[predictions[0],predictions[2],predictions[14],predictions[47],predictions[65]]

Cloning into 'fastcam'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 598 (delta 113), reused 130 (delta 70), pack-reused 414
Receiving objects: 100% (598/598), 18.54 MiB | 27.24 MiB/s, done.
Resolving deltas: 100% (353/353), done.
     |████████████████████████████████| 6.0MB 4.7MB/s 
  Created wheel for pytorch-gradcam: filename=pytorch_gradcam-0.2.1-cp36-none-any.whl size=5270 sha256=b678febc6b2dfe393050678cbb37d40b75c9d49a32ff8e46ac87140b3232acdf
  Stored in directory: /root/.cache/pip/wheels/e8/1e/35/d24150a078a90ce0ad093586814d4665e945466baa89907300
Successfully built pytorch-gradcam


#**Compute Saliency,Gradcam,Gradcam++ modules**

In [ ]:
# -*- coding: utf-8 -*-
%cd fastcam

#importing libraries 


from scipy import ndimage
from skimage.transform import resize
import yaml
import os
import torch
import warnings
import argparse
import tensorflow as tf
import math
import numpy as np
import torch
import torchvision
import torch.utils.data
from torch.utils.data import DataLoader,TensorDataset
import copy
import logging
from keras.layers import Input
from keras.layers.merge import concatenate
import torch.nn as nn
from torch.nn import BatchNorm3d,Conv3d,ReLU,MaxPool3d,Linear,AdaptiveAvgPool3d,Flatten,Softmax
import torch.nn.functional as F
from datetime import datetime
from torch.utils import data
import time

import skimage.io as sio
import shutil
from random import shuffle
from skimage.transform import resize
import skimage.io as sio
from scipy.io import savemat,loadmat
import cv2
import mask
import draw
import norm
import misc

from torchvision import models
from random import shuffle
from torchvision.utils import make_grid, save_image
import pandas as pd
from gradcam.utils import visualize_cam
from gradcam import GradCAMpp, GradCAM
from matplotlib import pyplot as plt





def get_smoe_map(x,relu=False):

  '''
  Scaled map order equivalent map computation fumction:
  reference from the saliency map paper : https://github.com/LLNL/fastcam.git and adapted/modified  to our usecase
  '''
  print(f' smoe input shape={x.shape}')
  if relu:
    x=tf.nn.relu(x).numpy()
  print(f'x range={np.amax(x),np.amin(x)}')
  
  m   = np.mean(x,axis=-1)+0.0000001 
  # m   = np.mean(x,axis=0)+0.0000001 #avoid log 0
  
  x   = x + 0.0000001
  # k   = np.log2(m) - np.mean(np.log2(x), axis=0)
  k   = np.log2(m) - np.mean(np.log2(x), axis=-1)
  print(f'log of mean={np.log2(m)}, mean of log={np.mean(np.log2(x), axis=-1)}')
  print(f'k={k}')
  k   = k + 0.0000001
  # k   = np.log10(m) - np.mean(np.log10(x), axis=-1)
  print(np.array_equal(np.zeros(k.shape),k))
  print(f'{x.shape,k.shape,np.amin(k)}')
  print(f'kmax, kmin={np.min(k),np.max(k)}')
  print(f'mean={m}')
  th  = k * m
  print(f'smoe map={th}')
  print(f'smoe output shape={th.shape}')
  return th

def get_std_map(x):
  '''
  STD based map alternative to SMOE.
  '''
  print(f'before std map shape ={x.shape}')
  m = np.std(x,axis=-1)

  print(f'std map shape ={m.shape}')

  return m

def get_norm(x,const_mean=None,const_std=None):
  '''
  get norm refrence from the saliency map paper : https://github.com/LLNL/fastcam.git and adapted/modified  to our usecase
  '''
  s0      = x.shape[0]
  s1      = x.shape[1]
  s2      = x.shape[2]

  # x       = np.reshape(x,(1,s0*s1))
  x       = np.reshape(x,(1,s1*s2))
  print(f'get norm func x after reshape={x.shape} ')

  '''
      Compute Mean
  '''
  if const_mean is None:
      m       = np.mean(x,axis=1)
      m       = np.reshape(m,(m.shape[0],1))
  else:
      m       = const_mean

  print(f'get norm func x after mean reshape={m.shape} ') 
  '''
      Compute Standard Deviation
  '''
  if const_std is None:
      s       = np.std(x,axis=1)
      s       = np.reshape(s,(s.shape[0],1))
  else:
      s       = const_std
  
  '''
      The normal cumulative distribution function is used to squash the values from within the range of 0 to 1
  '''

  s=torch.tensor(s)
  x       = 0.5*(1.0 + torch.erf((x-m)/(s*torch.sqrt(torch.tensor(2.0)))))
  print(x.shape)    
  # x       = x.reshape(1,s0,s1)
  x       = x.reshape(1,s1,s2)

  print(f'map after norm={x,x.shape}')
  return x


def combine_sal_maps(smaps,output_size,weights,map_num,resize_mode='bilinear',do_relu=False):
  '''
  Combined saliency maps are computed here .
  '''
  bn  = smaps[0].shape[0]
  cm  = torch.zeros((bn, 1, output_size[0], output_size[1]), dtype=smaps[0].dtype, device=smaps[0].device)
  ww  = []
  
  '''
      Now get each saliency map and resize it. Then store it and also create a combined saliency map.
  '''
  for i in range(len(smaps)):
      # assert torch.is_tensor(smaps[i]), "Each saliency map must be a Torch Tensor."
      wsz = smaps[i].shape
      w   = np.reshape(smaps[i],(wsz[0], 1, wsz[1], wsz[2]))#smaps[i].reshape(wsz[0], 1, wsz[1], wsz[2])
   
      w   = nn.functional.interpolate(w, size=output_size, mode=resize_mode, align_corners=False) 
      ww.append(w)        # should we weight the raw maps ... hmmm
      
      cm  += (w * weights[i])

  '''
      Finish the combined saliency map to make it a weighted average.
  '''
  weight_sum =sum(weights)
  cm  = cm / weight_sum
  cm  = cm.reshape(bn, output_size[0],output_size[1])
  
  ww  = torch.stack(ww,dim=1)
  ww  = ww.reshape(bn, map_num, output_size[0], output_size[1])
  

  
  return cm, ww

def compute_saliency_tf(base_path,inputs,tf_model):
  '''
   Saliency maps are computed for specicied layers and then combined
  '''

  gender=inputs[1]
  gender=tf.reshape(gender,[1,1])
  img=inputs[0]
  img_chunk=tf.convert_to_tensor(img)
  print(img_chunk.shape)
  img_chunk = tf.reshape(img_chunk,[1,121,145,6])
  layers=[layer.name for layer in tf_model.layers]
  outputs=[]

  #select all layer activations after conv for eg: if there ae 66 conv layers then there are 66 activation layers.
  for l in layers:
   
    if l.startswith('activation'):
    
        outputs.append(tf_model.get_layer(name=l).output) 

  outputs.append(tf_model.output)                                         
  test_tf_model=tf.keras.models.Model([tf_model.inputs], outputs)
  # test_tf_model=tf.keras.models.Model([tf_model.inputs], [tf_model.get_layer(name=layers[2]).output,tf_model.get_layer(name=layers[6]).output,tf_model.get_layer(name=layers[9]).output, tf_model.output])

  predictions = test_tf_model([img_chunk,gender])

  # Specify or experiment  with layers we want to compute saliency maps for.

  # hooks=[predictions[0],predictions[1],predictions[2],predictions[8],predictions[14],predictions[20],predictions[23]\
  #        ,predictions[29],predictions[35],predictions[41],predictions[47],predictions[50],\
  #        predictions[56],predictions[62],predictions[65]]#predictions[:layer_end]
  # hooks= [predictions[0],predictions[2],predictions[17],predictions[47],predictions[62]] 


  #these layers were picked as the outputs have diffrent scale dimensions,  we can experimentwith other layers as well.
  hooks=[predictions[0],predictions[2],predictions[14],predictions[47],predictions[65]] 
  
  # choose specific channels / filters
  for x in hooks:
    print('ouput shapes layerwise')
    print(x.shape)

  
  # sal_maps       = [ get_norm(get_smoe_map(np.expand_dims(np.mean(x.numpy()[:,:,:,:,:],axis=-2)[:,:,:,2],axis=-1))) for x in hooks ]

  #smoe
  sal_maps       = [ get_norm(get_smoe_map(np.mean(x.numpy()[:,:,:,:,:],axis=-2))) for x in hooks ]

  #std dev
  # sal_maps       = [ get_norm(get_std_map(np.mean(x.numpy()[:,:,:,:,:],axis=-2))) for x in hooks ]

  # sal_maps       = [ get_norm(get_smoe_map(x.numpy()[:,:,:,:,:])) for x in hooks ]
  for smaps in sal_maps:
    print(smaps.shape)

  # sal_maps = [np.reshape(smaps.numpy(),(121,121)) for smaps in sal_maps]

  # all layer scale maps with equal weightage
  weights=np.ones(len(hooks))
  
  # all layer scale maps with progressive increasing weightage
  # weights=[i+1 for i in range(len(hooks))]
  # weights = [i for i in range(len(hooks),0,-1)]
  
  map_num=len(hooks)

  f, axarr = plt.subplots(1,1,figsize=(10,10))
  raw=np.mean(img_chunk[0,:,:,:],axis=-1)
  raw= raw/np.max(raw)
  r=axarr.imshow(raw,cmap='jet')
  axarr.set_title('Input image mean along 3rd dimension')
  plt.colorbar(r,fraction=0.01, pad=0.04)
  plt.savefig(base_path+'mean_input_chunk.png')

  csal_maps,sal_maps = combine_sal_maps(sal_maps,output_size=[in_height,in_width],weights=weights,map_num=map_num)
  output_path = base_path +'Map_Combined.png'
  f, axarr = plt.subplots(1,1,figsize=(10,10))
  csal_map=csal_maps[0,:,:].numpy()
  imcs=csal_map/np.max(csal_map)
  im = axarr.imshow(imcs,cmap='jet')
  axarr.set_title('Combined saliency map')
  plt.colorbar(im,fraction=0.01, pad=0.04)
  plt.savefig(output_path)

  il = [sal_maps[0,i,:,:] for i in range(map_num)] # Put each saliency map into the figure
  il.append(csal_maps[0,:,:])                       # add in the combined map at the end of the figure
  images        = [torch.stack(il, 0)]          
  images        = make_grid(images, nrow=5)
  sal_img=images.unsqueeze(1)
  output_path=base_path +'Sal_Maps.png'
  save_image(sal_img,output_path)

  input_path = output_path
  f, axarr = plt.subplots(1,1,figsize=(10,10))
  im=sio.imread(input_path)
  im=axarr.imshow(np.mean(im,axis=-1)/255, cmap='jet');
  axarr.set_title('layerwise saliency maps')
  plt.colorbar(im,fraction=0.01, pad=0.04)
  output_path=base_path +'Sal_Maps_jet.png'
  plt.savefig(output_path)
  return csal_maps



def get_grads(layer_name,tf_model,inputs):
  '''
  computes gradients for GCAM/GCAM++
  '''

  cam_list=[]
  gender= inputs[1] #tf.constant([[1]],dtype=tf.float32)
  gender=tf.reshape(gender,[1,1])
  img=inputs[0]
  grad_model = tf.keras.models.Model([tf_model.inputs], [tf_model.get_layer(name=layer_name).output, tf_model.output])
  # for i in range(48,54,6):
  # end=start+6
  # print(f'{start,end}')
  # img_chunk2=
  img_chunk=tf.convert_to_tensor(img)
  img_chunk = tf.reshape(img_chunk,[1,121,145,6])

  cdr_ohe_dict={0:[1.0,0.0,0.0,0.0],0.5:[0.0,1.0,0.0,0.0],1:[0.0,0.0,1.0,0.0],2:[0.0,0.0,0.0,1.0]}
  cdr_keys= list(cdr_ohe_dict.keys())
  with tf.GradientTape() as tape:
      conv_outputs, predictions = grad_model([img_chunk,gender])
      print(f'predictions={predictions}')
      # loss=predictions
      loss = predictions[0] 
     

  output = conv_outputs[0]#[0,:,:,:,100]
  print(f'entering tape gradients')

  grads = tape.gradient(loss, conv_outputs)[0]#[0,:,:,:,100]
  print(type(grads))
  print(f'Crossed tape gradients')
  gate_f = tf.cast(output > 0, 'float32')
  gate_r = tf.cast(grads > 0, 'float32')
  # now there are 2 choice either use grads(raw grads) or use guided grads)
  guided_grads = tf.cast(output > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

  print(f'Entering reduce mean using guided_grads with shape={guided_grads.shape}')
  #guided grads
  weights = tf.reduce_mean(guided_grads, axis=(0,1,2))


  print(f'Computing CAM using output with shape:{output.shape}')

  print(f'weights={weights.shape}')
  cam = np.zeros(output.shape[0:3], dtype=np.float32)
  print(cam.shape)


  cam=tf.reduce_sum(tf.multiply(output,weights),axis=-1)
  cam_list.append(cam)
  return cam_list,grads,loss,weights,output,img_chunk




def compute_gcam_and_gcam_pp(layer_name,model,inputs):
  '''
  Generates GCAM/GCAM++
  '''
  cam_list,grads,loss,weights,output,img_chunk = get_grads(layer_name,model,inputs)
  
  heatmap_list=[]
  for i,cam in enumerate(cam_list):#as we are doing chunk wise so this camlist will have only one cam

    print(f'cam shape={cam.shape}')
    
    #gcam
    cam_map=resize(cam,(img_chunk.shape[1],img_chunk.shape[2],img_chunk.shape[3]))

    cam_map = np.maximum(cam_map,0)
    original_image=img_chunk.numpy()
   
    heatmap = (cam_map - cam_map.min()) / (cam_map.max() - cam_map.min())

  
    print(original_image.shape)
    image=np.mean(original_image[0,:,:,:],axis=-1)
    print(image.shape)

    mri_img=image#np.squeeze(image)
    heatmap_list.append(heatmap)
    # print(heatmap.min(),heatmap.max())

    heatmap_gcam = (cam_map - cam_map.min()) / (cam_map.max() - cam_map.min())

      
      
    gcam_img=(np.mean(heatmap_gcam,axis=-1)* 255).astype("uint8")
   
    #gcam++
    print(f'grads shape ={grads.shape},tf.exp(loss) shape={tf.exp(loss).shape}')
    conv_first_grad = tf.exp(loss)[0]*grads
    #second_derivative
    conv_second_grad = tf.exp(loss)[0]*grads*grads
    #triple_derivative
    conv_third_grad = tf.exp(loss)[0]*grads*grads*grads
    
    global_sum = np.sum(tf.reshape(output,(-1,conv_first_grad[0].shape[2])), axis=0)
    print(f'conv_first_grad shape={conv_first_grad.shape},conv_second_grad shape={conv_second_grad.shape} ,  conv_third_grad shape={conv_third_grad.shape}, global_sum.shape={global_sum.shape}  ')
    alpha_num = conv_second_grad[0]
    # alpha_denom = conv_second_grad[0]*2.0 + conv_third_grad[0]*global_sum.reshape((1,1,conv_first_grad[0].shape[2]))
    alpha_denom = conv_second_grad*2.0 + conv_third_grad*global_sum.reshape((1,1,1,conv_first_grad[0].shape[2]))
    alpha_denom = np.where(alpha_denom != 0.0, alpha_denom, np.ones(alpha_denom.shape))
    alphas = alpha_num/alpha_denom

    

    alphas_thresholding = np.where(weights, alphas, 0.0)
    print(f'alphas_thresholding shape={alphas_thresholding.shape}')
    alpha_normalization_constant = np.sum(np.sum(alphas_thresholding, axis=0),axis=0)
    alpha_normalization_constant_processed = np.where(alpha_normalization_constant != 0.0, alpha_normalization_constant, np.ones(alpha_normalization_constant.shape))
    print(f'alpha_normalization_constant_processed shape={alpha_normalization_constant_processed.shape}')
    
    # alphas /= alpha_normalization_constant_processed.reshape((1,1,conv_first_grad[0].shape[2]))
    alphas /= alpha_normalization_constant_processed.reshape((1,1,3,conv_first_grad[0].shape[2]))
    print(f'weights.shape={weights.shape},alphas.shape={alphas.shape}')
    weights_alpha=tf.reduce_sum(tf.multiply(weights,alphas),axis=0)
    
    cam=tf.reduce_sum(tf.multiply(output,weights_alpha),axis=-1)
    
    cam_map=resize(cam,(img_chunk.shape[1],img_chunk.shape[2],img_chunk.shape[3]))
  
    
    print(f'cam_map={cam_map.shape}')
    cam_map = np.maximum(cam_map, 0)

    heatmap_gcam_pp = (cam_map - cam_map.min()) / (cam_map.max() - cam_map.min())


    gcam_pp_img=(np.mean(heatmap_gcam_pp,axis=-1) * 255).astype("uint8")
    
    print(img_chunk.shape,mri_img.shape,gcam_img.shape,type(mri_img),type(gcam_img))
  
        
    return image, gcam_img,gcam_pp_img

def combine_sal_gcam(base_path,csmap,gcam_img,gcam_pp_img,image,layer_name,angle,result_path ):

  '''

  3 kinds of map computed : saliency map , saliency map combined  with GCAM, saliency map combined with GCAM++ 
  '''
  print(gcam_img.shape,csmap.shape,gcam_pp_img.shape,image.shape)
  #saliency map

  raw_tensor=torch.from_numpy(image).unsqueeze(0)
  heatmap_csmap, result_csmap = visualize_cam(csmap, raw_tensor)
  getMask                 = mask.SaliencyMaskDropout(keep_percent = 0.1, scale_map=False)
  hard_masked_csmap,_       = getMask(raw_tensor.unsqueeze(0),csmap)#.squeeze(0))
  hard_masked_csmap        = hard_masked_csmap.squeeze(0)
  masked_csmap             = misc.AlphaMask(raw_tensor, csmap.squeeze(0)).squeeze(0)
  

  vmin=0
  vmax=1.0
  f, axarr = plt.subplots(2,3,figsize=(20,20))
  img_plot = axarr[0][0].imshow(torch.mean(raw_tensor,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0][0].set_title('input')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[0][0])
  cbar.set_clim(0,1)
  img_plot = axarr[0][1].imshow(torch.mean(csmap,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0][1].set_title('combined saliency map')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[0][1])
  cbar.set_clim(0,1)
  img_plot = axarr[0][2].imshow(torch.mean(heatmap_csmap,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0][2].set_title('saliency map')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[0][2])
  cbar.set_clim(0,1)
  img_plot = axarr[1][0].imshow(torch.mean(result_csmap,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1][0].set_title('saliency map with alpha blend')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[1][0])
  cbar.set_clim(0,1)
  img_plot = axarr[1][1].imshow(masked_csmap,vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1][1].set_title('mask')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[1][1])
  cbar.set_clim(0,1)
  img_plot = axarr[1][2].imshow(hard_masked_csmap[0],vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1][2].set_title('hard mask')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[1][2])
  cbar.set_clim(0,1)
  plt.savefig(base_path+'saliency_only_fig.png')

  print(hard_masked_csmap.permute([2,0,1]).shape)
  csmap_img = torch.mean(csmap,axis=0).numpy()
  output_path   = base_path+"csmap.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':csmap_img ,'shape':csmap_img.shape})
  output_path   = base_path+"heatmap_csmap.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':heatmap_csmap.permute([1,2,0]).numpy() ,'shape':heatmap_csmap.permute([1,2,0]).numpy().shape})
  output_path   = base_path+"result_csmap.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':result_csmap.permute([1,2,0]).numpy() ,'shape':result_csmap.permute([1,2,0]).numpy().shape})
  output_path   = base_path+"hard_masked_csmap.png" 
  savemat(output_path.split('.png')[0] +'.mat',{'data':hard_masked_csmap.permute([1,2,0]).numpy() ,'shape':hard_masked_csmap.permute([1,2,0]).shape})
  output_path   = base_path+"masked_csmap.mat" 
  savemat(output_path,{'data':masked_csmap.numpy() ,'shape':masked_csmap.numpy().shape})
  masked_csmap_mat = loadmat(output_path)['data']
  plt.clf()
  p=plt.imshow(masked_csmap_mat,cmap='jet')
  plt.colorbar(p)      
  plt.clim(0.8,1)
  output_path   = base_path+"masked_csmap_0.8.png" 
  plt.savefig(output_path)

  #gcam

  if np.max(gcam_img) ==0:
    gcam_img = gcam_img+0.0000001
  if np.max(gcam_pp_img) ==0:
    gcam_pp_img = gcam_pp_img+0.0000001
  gcam_img_tensor=torch.from_numpy(gcam_img).unsqueeze(0)
  mask_gcam = csmap*(gcam_img_tensor)
  mask_gcam=mask_gcam/mask_gcam.max()
  

  

  heatmap_gcam, result_gcam = visualize_cam(mask_gcam, raw_tensor)
  getMask                 = mask.SaliencyMaskDropout(keep_percent = 0.1, scale_map=False)
  hard_masked_gcam,_       = getMask(raw_tensor.unsqueeze(0),mask_gcam)#.squeeze(0))
  hard_masked_gcam        = hard_masked_gcam.squeeze(0)
  masked_gcam             = misc.AlphaMask(raw_tensor, mask_gcam.squeeze(0)).squeeze(0)
  mx= str(np.max(masked_gcam.numpy()))
  plt.imsave(base_path+'masked_gcam_unnormalized_{0}max.png'.format(mx),masked_gcam.numpy(),cmap='jet')
  # masked_gcam              = misc.RangeNormalize(masked_gcam)


  #gcam++
  gcam_pp_img_tensor=torch.from_numpy(gcam_pp_img).unsqueeze(0)
  mask_gcam_pp = csmap*(gcam_pp_img_tensor)
  mask_gcam_pp=mask_gcam_pp/mask_gcam_pp.max()
  raw_tensor=torch.from_numpy(image).unsqueeze(0)
  heatmap_gcam_pp, result_gcam_pp = visualize_cam(mask_gcam_pp, raw_tensor)

  hard_masked_gcam_pp,_       = getMask(raw_tensor.unsqueeze(0),mask_gcam_pp)#.squeeze(0))
  hard_masked_gcam_pp         = hard_masked_gcam_pp.squeeze(0)
  masked_gcam_pp           = misc.AlphaMask(raw_tensor, mask_gcam_pp.squeeze(0)).squeeze(0)
  mx= str(np.max(masked_gcam_pp.numpy()))
  plt.imsave(base_path+'masked_gcam_pp_unnormalized_{0}max.png'.format(mx),masked_gcam_pp.numpy(),cmap='jet')
  # masked_gcam_pp           = misc.RangeNormalize(masked_gcam_pp) # avoid this step as it will normalize to 0 to 1 hence not good while comparing multiple scans


  #
  output_path   = base_path+"raw_img.mat"
  savemat(output_path,{'data':raw_tensor.numpy() ,'shape':raw_tensor.shape})
  background_img=loadmat(output_path)['data']

  base_path+='_'+layer_name

  #save gcam and gcam++ fig
  vmin=np.amin([np.min(gcam_img),np.min(gcam_pp_img)])
  vmax=np.amax([np.max(gcam_img),np.max(gcam_pp_img)])

  f, axarr = plt.subplots(1,2,figsize=(10,10))
  img_plot = axarr[0].imshow(gcam_img,vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0].set_title('Gradcam')
  img_plot = axarr[1].imshow(gcam_pp_img,vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1].set_title('Gradcam++')
  plt.colorbar(img_plot,fraction=0.046, pad=0.04)
  plt.savefig(base_path+'gcam_gcam++_fig.png')
  
  
  #gcam

  # vmin=np.amin([torch.min(raw_tensor),torch.min(csmap),torch.min(heatmap_gcam),torch.min(result_gcam),torch.min(masked_gcam),torch.min(hard_masked_gcam)])
  # vmax=np.amax([torch.max(raw_tensor),torch.max(csmap),torch.max(heatmap_gcam),torch.max(result_gcam),torch.max(masked_gcam),torch.max(hard_masked_gcam)])
  vmin=0
  vmax=1.0
  f, axarr = plt.subplots(2,3,figsize=(20,20))
  img_plot = axarr[0][0].imshow(torch.mean(raw_tensor,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0][0].set_title('input')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[0][0])
  cbar.set_clim(0,1)
  img_plot = axarr[0][1].imshow(torch.mean(csmap,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0][1].set_title('combined saliency map')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[0][1])
  cbar.set_clim(0,1)
  img_plot = axarr[0][2].imshow(torch.mean(heatmap_gcam,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0][2].set_title('saliency map + gradcam')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[0][2])
  cbar.set_clim(0,1)
  img_plot = axarr[1][0].imshow(torch.mean(result_gcam,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1][0].set_title('saliency map+gradcam with alpha blend')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[1][0])
  cbar.set_clim(0,1)
  img_plot = axarr[1][1].imshow(masked_gcam,vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1][1].set_title('mask')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[1][1])
  cbar.set_clim(0,1)
  img_plot = axarr[1][2].imshow(hard_masked_gcam[0],vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1][2].set_title('hard mask')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[1][2])
  cbar.set_clim(0,1)
  plt.savefig(base_path+'sal+gcam_fig.png')

  print(hard_masked_gcam.permute([2,0,1]).shape)
  output_path   = base_path+"gcam_img.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':gcam_img ,'shape':gcam_img.shape})
  output_path   = base_path+"heatmap_gcam.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':heatmap_gcam.permute([1,2,0]).numpy() ,'shape':heatmap_gcam.permute([1,2,0]).numpy().shape})
  output_path   = base_path+"result_gcam.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':result_gcam.permute([1,2,0]).numpy() ,'shape':result_gcam.permute([1,2,0]).numpy().shape})
  output_path   = base_path+"hard_masked_gcam.png" 
  savemat(output_path.split('.png')[0] +'.mat',{'data':hard_masked_gcam.permute([1,2,0]).numpy() ,'shape':hard_masked_gcam.permute([1,2,0]).shape})
  output_path   = base_path+"masked_gcam.mat" 
  savemat(output_path,{'data':masked_gcam.numpy() ,'shape':masked_gcam.numpy().shape})
  masked_gcam_mat = loadmat(output_path)['data']
  plt.clf()
  p=plt.imshow(masked_gcam_mat,cmap='jet')
  plt.colorbar(p)      
  plt.clim(0.8,1)
  output_path   = base_path+"masked_gcam_0.8.png" 
  plt.savefig(output_path)
  
  #gcam_pp
  vmin=np.amin([torch.min(raw_tensor),torch.min(csmap),torch.min(heatmap_gcam_pp),torch.min(result_gcam_pp),torch.min(masked_gcam_pp),torch.min(hard_masked_gcam_pp)])
  vmax=np.amax([torch.max(raw_tensor),torch.max(csmap),torch.max(heatmap_gcam_pp),torch.max(result_gcam_pp),torch.max(masked_gcam_pp),torch.max(hard_masked_gcam_pp)])

  vmin=0
  vmax=1.0

  f, axarr = plt.subplots(2,3,figsize=(20,20))
  img_plot = axarr[0][0].imshow(torch.mean(raw_tensor,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0][0].set_title('input')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[0][0])
  cbar.set_clim(0,1)
  img_plot = axarr[0][1].imshow(torch.mean(csmap,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0][1].set_title('combined saliency map')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[0][1])
  cbar.set_clim(0,1)
  img_plot = axarr[0][2].imshow(torch.mean(heatmap_gcam_pp,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0][2].set_title('saliency map + gradcam++')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[0][2])
  cbar.set_clim(0,1)
  img_plot = axarr[1][0].imshow(torch.mean(result_gcam_pp,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1][0].set_title('saliency map+gradcam++ with alpha blend')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[1][0])
  cbar.set_clim(0,1)
  img_plot = axarr[1][1].imshow(masked_gcam_pp,vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1][1].set_title('mask')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[1][1])
  cbar.set_clim(0,1)
  img_plot = axarr[1][2].imshow(hard_masked_gcam_pp[0],vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1][2].set_title('hard mask')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[1][2])
  cbar.set_clim(0,1)
  plt.savefig(base_path+'sal+gcam++_fig.png')


  raw_img = torch.mean(raw_tensor,axis=0).numpy()
  output_path   = base_path+"raw_input.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':raw_img ,'shape':raw_img.shape})

  f, axarr = plt.subplots(1,1,figsize=(10,10))
  
  r=axarr.imshow(raw_img,cmap='gray')
  axarr.set_title('raw gray image')
  cbar=plt.colorbar(r,fraction=0.046, pad=0.04)
  cbar.set_clim(0,1)
  plt.savefig(base_path+'raw_gray_cbar.png')

  
  output_path   = base_path+"gcam_pp_img.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':gcam_pp_img ,'shape':gcam_pp_img.shape})
  output_path   = base_path+"heatmap_gcam_pp.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':heatmap_gcam_pp.permute([1,2,0]).numpy() ,'shape':heatmap_gcam_pp.permute([1,2,0]).numpy().shape})
  output_path   = base_path+"result_gcam_pp.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':result_gcam_pp.permute([1,2,0]).numpy() ,'shape':result_gcam_pp.permute([1,2,0]).numpy().shape})
  output_path   = base_path+"hard_masked_gcam_pp.png" 
  savemat(output_path.split('.png')[0] +'.mat',{'data':hard_masked_gcam_pp.permute([1,2,0]).numpy() ,'shape':hard_masked_gcam_pp.permute([1,2,0]).numpy().shape})
  output_path   = base_path+"masked_gcam_pp.mat" 
  savemat(output_path,{'data':masked_gcam_pp.numpy() ,'shape':masked_gcam_pp.numpy().shape})
  masked_gcam_pp_mat = loadmat(output_path)['data']
  plt.clf()
  # p=plt.imshow(masked_gcam_pp_mat,cmap='jet')
  # plt.colorbar(p)      
  # plt.clim(0.8,1)
  # output_path   = base_path+"masked_gcam_pp_0.8.png" 
  # plt.savefig(output_path)
  

  im=ndimage.rotate(masked_gcam_pp_mat,angle)
  max=1
  im = im/max #optional step as in our case max is 1 also use any contant val.

  im[im<0.3]=np.nan
  plt.imshow(im,cmap='jet')
  plt.axis('off')
  plt.clim(0,1)
  plt.savefig(result_path+'_result.png')

  

  '''
  These are the images used in the final results the ones inside result_path
  '''
  plt.clf() # clear existing figure
  #mask overlaid on gray matter
  print(f'background shape before={background_img.shape}')
  im2=background_img[0]
  print(f'background shape after={im2.shape}')
  im2=ndimage.rotate(im2,angle)
  im2=1-im2
  gray=plt.imshow(im2,cmap='gray')
  plt.axis('off')
  im=ndimage.rotate(masked_gcam_pp_mat,angle)
  im[im<0.3]=np.nan
  heat=plt.imshow(im,cmap='jet')
  plt.axis('off')
  plt.clim(0,1)
  plt.colorbar()
  plt.savefig(result_path+'_result_overlay.png')


 


  ## Saliency only
  masked_csmap=masked_csmap.numpy()

  t='masked_only_saliency'



  max =  1 #or use any other constant
  
  frac=0.3
  r1=(masked_csmap/max)

  r1[np.where(r1<frac*np.max(r1))]=0
  # r1[np.where(r1<np.median(r1))]=0
  plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet')
  

  # frac=0.5
  # r1=(masked_csmap/max)

  # r1[np.where(r1<frac*np.max(r1))]=0
  # # r1[np.where(r1<np.median(r1))]=0
  # plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet')

  

  # frac=0.8
  # r1=(masked_csmap/max)

  # r1[np.where(r1<frac*np.max(r1))]=0
  # # r1[np.where(r1<np.median(r1))]=0
  # plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet')


  ## GCAM
  masked_gcam=masked_gcam.numpy()

  

  t='masked_gcam'

  max =  1 #np.amax(masked_gcam)

  frac=0.3
  r1=(masked_gcam/max)

  r1[np.where(r1<frac*np.max(r1))]=0
  # r1[np.where(r1<np.median(r1))]=0
  plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet')


  # frac=0.5
  # r1=(masked_gcam/max)

  # r1[np.where(r1<frac*np.max(r1))]=0
  # # r1[np.where(r1<np.median(r1))]=0
  # plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet')

  

  # frac=0.8
  # r1=(masked_gcam/max)

  # r1[np.where(r1<frac*np.max(r1))]=0
  # # r1[np.where(r1<np.median(r1))]=0
  # plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet')

  ## GCAM++
  ## fraction mask  map for overlaying GCAM++
  masked_gcam_pp=masked_gcam_pp.numpy()

  

  t='masked_gcam_pp'
  max = 1 #np.amax(masked_gcam_pp)

  frac=0.3
  r1=(masked_gcam_pp/max)

  r1[np.where(r1<frac*np.max(r1))]=0
  # r1[np.where(r1<np.median(r1))]=0
  plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet')
  
  # frac=0.5
  # r1=(masked_gcam_pp/max)



  # r1[np.where(r1<frac*np.max(r1))]=0
  # # r1[np.where(r1<np.median(r1))]=0
  # plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet')

  

  # frac=0.8
  # r1=(masked_gcam_pp/max)

  # r1[np.where(r1<frac*np.max(r1))]=0
  # # r1[np.where(r1<np.median(r1))]=0
  # plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet')

  ##


/content/fastcam


# **Tf records loading and parsing utility**

In [ ]:
def get_selected_scan_from_subjects(data_path,subject_ids,label_df,selected_scans):
    '''
    Same as get_scan_from_subjects in case we need specific scan path and not all scans of a patient.
    '''
    scans=[]
    labels=[]
    gender=[]
    cdr=[]
    ids=[]
    subject_ids = set(subject_ids)
    for subject in subject_ids :
        path=os.path.join(data_path,subject)
        paths=os.listdir(path)

        ids.extend([scan_id.split('.')[0] for scan_id in paths  if scan_id.split('/')[-1].split('.')[0] in selected_scans ])
        scans.extend([ os.path.join(path,scan_id) for scan_id in paths   if scan_id.split('/')[-1].split('.')[0] in selected_scans ])
        
    
        labels.extend([label_df[label_df['MRI ID']==scan_id.split('.')[0]]['Age'].to_list()[0] for scan_id in paths   if scan_id.split('/')[-1].split('.')[0] in selected_scans ])
        gender.extend([label_df[label_df['MRI ID']==scan_id.split('.')[0]]['M/F'].to_list()[0] for scan_id in paths   if scan_id.split('/')[-1].split('.')[0] in selected_scans])
        cdr.extend([label_df[label_df['MRI ID']==scan_id.split('.')[0]]['CDR'].to_list()[0] for scan_id in paths  if scan_id.split('/')[-1].split('.')[0] in selected_scans])

    return scans,labels,gender,ids,cdr

def get_scan_from_subjects(data_path,subject_ids,label_df):
    '''
    Scan paths,cdr,gender,labels  of subjects .
    '''
    scans=[]
    labels=[]
    gender=[]
    cdr=[]
    ids=[]
    subject_ids = set(subject_ids)
    for subject in subject_ids :
        path=os.path.join(data_path,subject)
        paths=os.listdir(path)
        ids.extend([scan_id.split('.')[0] for scan_id in paths ])
        scans.extend([ os.path.join(path,scan_path) for scan_path in paths  ])
 
        labels.extend([label_df[label_df['MRI ID']==scan_id.split('.')[0]]['Age'].to_list()[0] for scan_id in paths   ])
        gender.extend([label_df[label_df['MRI ID']==scan_id.split('.')[0]]['M/F'].to_list()[0] for scan_id in paths  ])
        cdr.extend([label_df[label_df['MRI ID']==scan_id.split('.')[0]]['CDR'].to_list()[0] for scan_id in paths ])
    #print(labels,gender)
           
    # shuffle(scans)
    
    return scans,labels,gender,ids,cdr


def get_test_files(label_path,data_path,debug_mode_subject=None,selected_scans=[]):
    '''
    Primary function to get scans.
    '''

    data = pd.read_csv(label_path)
    data = data.rename(columns={'MR ID':'MRI ID'})
    data['M/F'] = encode_gender(data)
    if debug_mode_subject is None:
      test_ids = os.listdir(data_path)
    else:
      test_ids=debug_mode_subject
    
    shuffle(test_ids)
    if len(selected_scans)>0:
      test_patients,test_labels,test_gender,scan_ids,test_cdr = get_selected_scan_from_subjects(data_path,test_ids,data,selected_scans)
    else:
      test_patients,test_labels,test_gender,scan_ids,test_cdr = get_scan_from_subjects(data_path,test_ids,data)
   
    return test_patients,scan_ids, test_labels,test_gender,test_cdr
   
def encode_gender(data):
    '''
    Categorical encoding. for gender.
    '''
    data['M/F'] = pd.Categorical(data['M/F'])
    
    return data['M/F'].cat.codes

def parse_function_image(example_proto):

    features = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'image_shape': tf.io.FixedLenFeature([], tf.string)
    }

    content = tf.io.parse_single_example(example_proto, features=features)

    content['image_shape'] = tf.io.decode_raw(content['image_shape'], tf.int32)
    content['image'] = tf.io.decode_raw(content['image'], tf.float32)
    content['image'] = tf.reshape(content['image'], content['image_shape'])

    return content['image']


In [ ]:
# dt_string_old=dt_string
# dt_string

exp='exp_ba'
healthy_path = '/content/drive/My Drive/BA_Estimation/models/{0}/testset.csv'.format(exp)
# ad_path = '/content/drive/My Drive/BA_Estimation/models/{0}/outlier.csv'.format(exp)
hdf = pd.read_csv(healthy_path)
# adf = pd.read_csv(ad_path)
# print(hdf.columns)

sub =  hdf['patient_id'].values.tolist()
# sub =  adf['patient_id'].values.tolist()
# sub.extend(adf['patient_id'].values.tolist())
sub= list(set(sub))
scans= copy.deepcopy(sub)
for i,s in enumerate(sub) :
  if s.startswith('OAS1'):
    s= s[:9] #OAS1_0123_MR1 take first 9 characters
    sub[i] = s.replace('_','')
  elif s.startswith('OAS3'): #OAS31098_MR_d7178 #take just subject id
    sub[i] = s.split('_')[0]
# debug_mode_subject=None
print(f'subjects={sub}')
print(f'scans={scans}')

subjects=['OAS30683', 'OAS31049', 'OAS30058', 'OAS30099', 'OAS30046', 'OAS30835', 'OAS30966', 'OAS30734', 'OAS30240', 'OAS30711', 'OAS30751', 'OAS30676', 'OAS30007', 'OAS30741', 'OAS30167', 'OAS30778', 'OAS30168', 'OAS30144', 'OAS30919', 'OAS31142', 'OAS30048', 'OAS30765', 'OAS31111', 'OAS30109', 'OAS30070', 'OAS30744', 'OAS31043', 'OAS30582', 'OAS31012', 'OAS30466', 'OAS30048', 'OAS30822', 'OAS31092', 'OAS30729', 'OAS30019', 'OAS31068', 'OAS31142', 'OAS31086', 'OAS30627', 'OAS30778', 'OAS30178', 'OAS31118', 'OAS30496', 'OAS30539', 'OAS30930', 'OAS30866', 'OAS30092', 'OAS31001', 'OAS30392', 'OAS30240', 'OAS30625', 'OAS30906', 'OAS30006', 'OAS30700', 'OAS30883', 'OAS31038', 'OAS30063', 'OAS30775', 'OAS30333', 'OAS30534', 'OAS30184', 'OAS30731', 'OAS30312', 'OAS30295', 'OAS30327', 'OAS30616', 'OAS30866', 'OAS30372', 'OAS30194', 'OAS30852', 'OAS30459', 'OAS30685', 'OAS30954', 'OAS30431', 'OAS30776', 'OAS30025', 'OAS30914', 'OAS30449', 'OAS30517', 'OAS30109', 'OAS30936', 'OAS30184', 'OAS31

In [ ]:
len(scans),len(sub)

# **Visualize maps on BA estimation agenet model on testset**

In [ ]:
#load model

cf={'Pretrained_Model':{'path':'/content/drive/My Drive/BA_Estimation/models/exp_ba/age_net.hdf5'},'Paths':\
      {'labels':'/content/drive/My Drive/BA_Estimation/csv_data/oasis1_oasis3_labels.csv',\
       'test_tfrecord':'/content/drive/My Drive/BA_Estimation/tf_records_data/testing_all_cdr'}}#testing_all_cdr
model =  tf.python.keras.models.load_model(cf['Pretrained_Model']['path'],compile=False)


In [ ]:

print(tf.__version__)


# model =  tf.python.keras.models.load_model(cf['Pretrained_Model']['path'],compile=False)
tf_model = model
tfw=tf_model.weights

dt_string = datetime.now().strftime('%d-%m-%Y-%H-%M')+'_smoe_maps_blockend_scale_endlayers_equal_weights'
# dt_string = '01-09-2020-12-33_smoe_maps_blockend_scale_endlayers_equal_weights'
# dt_string = '07-09-2020-15-56_smoe_maps_blockend_scale_endlayers_equal_weights'

label_path= cf['Paths']['labels'] #'/content/drive/My Drive/BA_Estimation/csv_data/oasis1_oasis3_labels.csv'#/media/shashanks/My Passport/documents/Master_Thesis_Backup/data/
data_path= cf['Paths']['test_tfrecord'] #'/home/raarmak1/.shashanks/BA_estimation/tfrecords_data/OASIS1_3_combined/tfrecords_data_exp3/testing/'
# exp='exp_ba_outliers'

# debug_mode_subject= ['OAS31054']#
# debug_mode_subject = ['OAS30884','OAS30720','OAS30344','OAS30541'] #,'OAS31054'] #OAS30911
#outliers
# debug_mode_subject=['OAS30736','OAS30282','OAS30190']#
# debug_mode_subject=['OAS31156','OAS30377','OAS30622','OAS30091','OAS30440','OAS30947','OAS30126','OAS30071','OAS31002','OAS30194','OAS30379','OAS31090',]
# debug_mode_subject=['OAS30628','OAS30102','OAS30194','OAS30383','OAS30126','OAS31002']#'OAS30955','OAS30955','OAS30457','OAS30849','OAS30110','OAS30941',]
#healthy
# debug_mode_subject=['OAS31035','OAS30310','OAS30392','OAS30512','OAS30273','OAS31153','OAS30956']

exp='exp_ba'
healthy_path = '/content/drive/My Drive/BA_Estimation/models/{0}/non_outliers.csv'.format(exp) # '/content/drive/My Drive/BA_Estimation/models/{0}/testset.csv'.format(exp)
# ad_path = '/content/drive/My Drive/BA_Estimation/models/{0}/outlier.csv'.format(exp)
hdf = pd.read_csv(healthy_path)
# adf = pd.read_csv(ad_path)
# print(hdf.columns)

sub =  hdf['patient_id'].values.tolist()
# sub =  adf['patient_id'].values.tolist()
# sub.extend(adf['patient_id'].values.tolist())
sub= list(set(sub))
scans= copy.deepcopy(sub)
for i,s in enumerate(sub) :
  if s.startswith('OAS1'):
    s= s[:9] #OAS1_0123_MR1 take first 9 characters
    sub[i] = s.replace('_','')
  elif s.startswith('OAS3'): #OAS31098_MR_d7178 #take just subject id
    sub[i] = s.split('_')[0]
# debug_mode_subject=None
print(f'subjects={sub}')
print(f'scans={scans}')
test_patients,scan_ids, test_labels,test_gender,test_cdr = get_test_files(label_path,data_path,debug_mode_subject=sub[100:200],selected_scans=scans[100:200])
# test_patients,scan_ids, test_labels,test_gender,test_cdr = get_test_files(label_path,data_path,debug_mode_subject)
tfr=tf.data.TFRecordDataset(test_patients)
img_tf=tfr.map(map_func=lambda a:parse_function_image(a))

case='healthy'
# case = 'outliers'
exp='exp_ba_healthy'
# exp='exp_ba_outliers'
gender_dict={0:'Female',1:'Male'}
counter =0
for i,im in enumerate(img_tf): 
    print(type(im),im.shape)
    if test_cdr[i] > 0: #if block to only inclue healthy subjects from testset for visualizations
      continue 
    counter+=1

    img=im.numpy()
    print(img.shape)
  
    max_intensity=0
    csmap_list=[]
    for chunk_id in [6,7,9]:# [7,9,11]: #[7,8,9,10,11,12,13,14]:#[3,7,9,11,14,18]:#range(1,21):

      start = (chunk_id-1)*6
      end = chunk_id*6
      
      img_chunk=torch.tensor(img[:,:,start:end])
      # img_chunk=torch.tensor(img[:,:,0:6])
      img_chunk = img_chunk.unsqueeze(0)
      input_tensor = img_chunk.unsqueeze(0)
      # imshow(input_tensor[0,0,:,:,-1],cmap='jet')
      in_height   = input_tensor.size()[2]
      in_width    = input_tensor.size()[3]
      print(test_gender[i],input_tensor.shape,scan_ids[i])
      # break
      n=66
      # layer_name='conv3d_'+str(n-1)
      # layer_name='activation_'+str(n-1)
      layer_name='activation_'+str(n)
      # conv_path ='conv1_'+str(n)+'_tfw'
      # conv_path ='conv1_'+str(n)+'_blockend_scale_endlayers_equal_weights'
      conv_path ='conv1_'+str(n)
      
      cdr_val = str(test_cdr[i])
      #axial
      base_dir= '/content/drive/My Drive/BA_Estimation/results/sal_map_axial/'+dt_string\
      +'_'+exp+'/'+scan_ids[i]+'_cdr'+str(test_cdr[i])
      path = base_dir+'/'+str(chunk_id)+'_'+conv_path+'/'
      print(path)
      if not os.path.exists(path):
        os.makedirs(path)
      
      base_path  = path+scan_ids[i]+'_chunk_'+str(chunk_id)
      result_path = '/content/drive/My Drive/BA_Estimation/final_results/{0}/'.format(case)+dt_string+'/cdr'+cdr_val+ '/axial/'+str(chunk_id)+'/'

      if not os.path.exists(result_path):
        os.makedirs(result_path)
      result_path += scan_ids[i]
      csmap_a=compute_saliency_tf(base_path,inputs=[img[:,:,start:end],test_gender[i]],tf_model=tf_model)
      image,gcam_img,gcam_pp_img = compute_gcam_and_gcam_pp(layer_name,tf_model,[img[:,:,start:end],test_gender[i]])
      combine_sal_gcam(path+scan_ids[i]+'_cdr'+str(test_cdr[i])+'_'+gender_dict[test_gender[i]],csmap_a,gcam_img,gcam_pp_img,image,layer_name=layer_name,angle=-270,result_path=result_path ) 
      
      

      #sagittal
      base_dir= '/content/drive/My Drive/BA_Estimation/results/sal_map_sagittal/'+dt_string\
      +'_'+exp+'/'+scan_ids[i]+'_cdr'+str(test_cdr[i])
      path = base_dir+'/'+str(chunk_id)+'_'+conv_path+'/'
      print(path)
      
      if not os.path.exists(path):
        os.makedirs(path)
      base_path  = path+scan_ids[i]+'_chunk_'+str(chunk_id)
      img_s= torch.from_numpy(img[start:end,:,:]).permute(2,1,0)
      result_path = '/content/drive/My Drive/BA_Estimation/final_results/{0}/'.format(case)+dt_string +'/cdr'+cdr_val+'/sagittal/'+str(chunk_id)+'/'
      if not os.path.exists(result_path):
        os.makedirs(result_path)
      result_path += scan_ids[i]
      csmap_s=compute_saliency_tf(base_path,inputs=[img_s.numpy(),test_gender[i]],tf_model=tf_model)
      image,gcam_img,gcam_pp_img = compute_gcam_and_gcam_pp(layer_name,tf_model,[img_s.numpy(),test_gender[i]])
      combine_sal_gcam(path+scan_ids[i]+'_cdr'+str(test_cdr[i])+'_'+gender_dict[test_gender[i]],csmap_s,gcam_img,gcam_pp_img,image,layer_name=layer_name,angle=180,result_path=result_path ) 

      #coronal
      base_dir= '/content/drive/My Drive/BA_Estimation/results/sal_map_coronal/'+dt_string\
      +'_'+exp+'/'+scan_ids[i]+'_cdr'+str(test_cdr[i])
      # path = base_dir+'/'+str(chunk_id)+'_old_'+conv_path+'/'
      result_path = '/content/drive/My Drive/BA_Estimation/final_results/{0}/'.format(case)+dt_string+'/cdr'+cdr_val+'/coronal/'+str(chunk_id)+'/'
      path = base_dir+'/'+str(chunk_id)+'_'+conv_path+'/'
      print(path)
      if not os.path.exists(result_path):
        os.makedirs(result_path)
      result_path += scan_ids[i]
      if not os.path.exists(path):
        os.makedirs(path)
      base_path  = path+scan_ids[i]+'_chunk_'+str(chunk_id)
      img_c= torch.from_numpy(img[:,start:end,:]).permute(2,0,1)
      img_c=img_c.unsqueeze(0)
      img_c = torch.nn.functional.upsample(img_c.unsqueeze(0), size=(121,145,6), mode='nearest') 
      csmap_c=compute_saliency_tf(base_path,inputs=[img_c.numpy(),test_gender[i]],tf_model=tf_model)
      image,gcam_img,gcam_pp_img = compute_gcam_and_gcam_pp(layer_name,tf_model,[img_c.numpy(),test_gender[i]])
      combine_sal_gcam(path+scan_ids[i]+'_cdr'+str(test_cdr[i])+'_'+gender_dict[test_gender[i]],csmap_c,gcam_img,gcam_pp_img,image,layer_name=layer_name,angle=180,result_path=result_path ) 


    print(f'scan count={counter}')

      # #coronal
      # base_dir= '/content/drive/My Drive/BA_Estimation/results/sal_map_coronal/'+dt_string\
      # +'_'+exp+'/'+scan_ids[i]+'_cdr'+str(test_cdr[i])
      # path = base_dir+'/'+str(chunk_id)+'_new_'+conv_path+'/'
      # print(path)
      # if not os.path.exists(path):
      #   os.makedirs(path)
      # base_path  = path+scan_ids[i]+'_chunk_'+str(chunk_id) #36:42
      # img_c= torch.from_numpy(img[:,42:48,:]).permute(2,0,1)
      # img_c=img_c.unsqueeze(0)
      # img_c = torch.nn.functional.upsample(img_c.unsqueeze(0), size=(121,145,6), mode='nearest') 
      # csmap_c=compute_saliency_tf(base_path,inputs=[img_c.numpy(),test_gender[i]],tf_model=tf_model)
      # image,gcam_img,gcam_pp_img = compute_gcam_and_gcam_pp(layer_name,tf_model,[img_c.numpy(),test_gender[i]])
      # combine_sal_gcam(path+scan_ids[i]+'_cdr'+str(test_cdr[i])+'_'+gender_dict[test_gender[i]],csmap_c,gcam_img,gcam_pp_img,image,layer_name=layer_name ) 
      

    


# **Visualizing mean testset on BA models.**
## The mean was computed on the test set ateach cdr value for male and female separately and then visualized .

In [ ]:

mat_path = '/content/drive/My Drive/BA_Estimation/means/exp_ba/full/'
files = os.listdir(mat_path)

# dt_string = datetime.now().strftime('%d-%m-%Y-%H-%M')+'_smoe_maps_blockend_scale_endlayers_equal_weights'

label_path= cf['Paths']['labels'] 
data_path= cf['Paths']['test_tfrecord'] 
# exp='exp_siam_ad'
exp='exp_ba_means'
tf_model = model

gender_dict={0:'Female',1:'Male'}
reverse_gender_dict = {'Female':0,'Male':0}
for i,f in enumerate(files): #OAS30686_d0030
  # print(type(f),im.shape)
  
  test_gender = f.split('_')[-1].split('.')[0]
  gender_tensor= np.array(reverse_gender_dict[test_gender])
  test_cdr = f.split('_')[-2][-3:]
  img=loadmat(mat_path+f)['data']
  print(img.shape)
  csmap_list=[]

  for chunk_id in [6,7]:#range(1,21): #select chunks from 1,21 (significant ones are from chunk 6-11)

    start = 6*(chunk_id-1)
    end = start+6
    # img = loadmat(f)['data']

    img_chunk=torch.tensor(img[:,:,start:end]).unsqueeze(0)
    img_chunk = img_chunk.unsqueeze(-1)
    
    print(f'input shape={img_chunk.shape}')
   
    input_tensor = img_chunk

    in_height   = input_tensor.size()[1]
    in_width    = input_tensor.size()[2]
    print(test_gender,input_tensor.shape,f)

    a=51
    n=35

    layer_name='activation_'+str(a)

    conv_path ='conv1_'+str(n)


    #axial
    base_dir= '/content/drive/My Drive/BA_Estimation/means/'+dt_string+ '/sal_map_axial/' \
    +exp+'/'+f+'_cdr'+str(test_cdr)
    path = base_dir+'/'+str(chunk_id)+'_'+conv_path+'/'
    print(path)
    if not os.path.exists(path):
      os.makedirs(path)
    base_path  = path+f+'_chunk_'+str(chunk_id)
    csmap_a=compute_saliency_tf(base_path,inputs=[input_tensor,gender_tensor],tf_model=tf_model)
    image,gcam_img,gcam_pp_img = compute_gcam_and_gcam_pp(layer_name,tf_model,[input_tensor,gender_tensor])
    combine_sal_gcam(path+f+'_cdr'+str(test_cdr)+'_'+test_gender,csmap_a,gcam_img,gcam_pp_img,image,layer_name=layer_name ) 


    #sagittal
    base_dir= '/content/drive/My Drive/BA_Estimation/means/'+dt_string+'/sal_map_sagittal/' \
    +exp+'/'+f+'_cdr'+str(test_cdr)
    path = base_dir+'/'+str(chunk_id)+'_'+conv_path+'/'
    print(path)
    if not os.path.exists(path):
      os.makedirs(path)
    base_path  = path+f+'_chunk_'+str(chunk_id)
    img_s= torch.from_numpy(img[start:end,:,:]).permute(2,1,0)
    csmap_s=compute_saliency_tf(base_path,inputs=[img_s,gender_tensor],tf_model=tf_model)
    image,gcam_img,gcam_pp_img = compute_gcam_and_gcam_pp(layer_name,tf_model,[img_s,gender_tensor])
    combine_sal_gcam(path+f+'_cdr'+str(test_cdr)+'_'+test_gender,csmap_s,gcam_img,gcam_pp_img,image,layer_name=layer_name ) 


    #coronal
    base_dir= '/content/drive/My Drive/BA_Estimation/means/'+dt_string + '/sal_map_coronal/' \
    +exp+'/'+f+'_cdr'+str(test_cdr)
    path = base_dir+'/'+str(chunk_id)+'_old_'+conv_path+'/'
    print(path)
    if not os.path.exists(path):
      os.makedirs(path)
    base_path  = path+f+'_chunk_'+str(chunk_id)
    # img_c= torch.from_numpy(img[:,58:64,:]).permute(2,0,1)
    img_c= torch.from_numpy(img[:,start:end,:]).permute(2,0,1)
    img_c=img_c.unsqueeze(0)
    img_c = torch.nn.functional.upsample(img_c.unsqueeze(0), size=(121,145,6), mode='nearest') 
    csmap_c=compute_saliency_tf(base_path,inputs=[img_c,gender_tensor],tf_model=tf_model)
    image,gcam_img,gcam_pp_img = compute_gcam_and_gcam_pp(layer_name,tf_model,[img_c,gender_tensor])
    combine_sal_gcam(path+f+'_cdr'+str(test_cdr)+'_'+test_gender,csmap_c,gcam_img,gcam_pp_img,image,layer_name=layer_name ) 

    #coronal
    base_dir= '/content/drive/My Drive/BA_Estimation/means/'+dt_string + '/sal_map_coronal/' \
    +exp+'/'+f+'_cdr'+str(test_cdr)
    path = base_dir+'/'+str(chunk_id)+'_new_'+conv_path+'/'
    print(path)
    if not os.path.exists(path):
      os.makedirs(path)
    base_path  = path+f+'_chunk_'+str(chunk_id)
    # img_c= torch.from_numpy(img[:,58:64,:]).permute(2,0,1)
    img_c= torch.from_numpy(img[:,end:end+6,:]).permute(2,0,1)
    img_c=img_c.unsqueeze(0)
    img_c = torch.nn.functional.upsample(img_c.unsqueeze(0), size=(121,145,6), mode='nearest') 
    csmap_c=compute_saliency_tf(base_path,inputs=[img_c,gender_tensor],tf_model=tf_model)
    image,gcam_img,gcam_pp_img = compute_gcam_and_gcam_pp(layer_name,tf_model,[img_c,gender_tensor])
    combine_sal_gcam(path+f+'_cdr'+str(test_cdr)+'_'+test_gender,csmap_c,gcam_img,gcam_pp_img,image,layer_name=layer_name ) 




    

# **End of code notebook (the following cells below are only for experimental trying outs.)**

In [ ]:
img=loadmat('/content/drive/My Drive/BA_Estimation/results/sal_map_axial/25-08-2020-11-36_smoe_maps_blockend_scale_endlayers_equal_weights_exp_ba/OAS30190_MR_d0082_cdr1.0/9_conv1_66/OAS30190_MR_d0082_cdr1.0_Female_activation_66masked_gcam_pp.mat')

img['data'].shape

(121, 145)

In [ ]:
image=img['data']
image[np.where(image<0.3*np.max(image))]=0
im=plt.imshow(image,cmap='jet')
plt.colorbar(im)
plt.clim(0.6,1)

In [ ]:
image=img['data']
image[np.where(image<0.3*np.max(image))]=0
im=plt.imshow(image,cmap='jet')
plt.colorbar(im)
plt.clim(0.6,0.9)

In [ ]:
import nibabel
import matplotlib.pyplot as plt
import numpy as np
# nii_img=nibabel.load('/content/sub-OAS30440_ses-d0163_T1w.nii').get_fdata() /content/OAS30070_MR_d0070.tfrecord
# nii_img=nibabel.load('/content/sub-OAS30070_ses-d0070_T1w.nii').get_fdata() /content/smwc1sub-OAS30282_ses-d0040_T1w.nii
# nii_img=nibabel.load('/content/smwc1sub-OAS30282_ses-d0040_T1w.nii').get_fdata() /content/sub-OAS31035_ses-d5659_run-02_T1w.nii
# nii_img=nibabel.load('/content/sub-OAS31035_ses-d5659_run-02_T1w.nii').get_fdata() /content/sub-OAS30310_ses-d0191_T1w.nii
# nii_img=nibabel.load('/content/sub-OAS31002_ses-d4948_run-02_T1w.nii').get_fdata()
# nii_img=nibabel.load('/content/sub-OAS30102_ses-d0024_T1w.nii').get_fdata()
# nii_img=nibabel.load('/content/sub-OAS30383_ses-d0134_run-02_T1w.nii').get_fdata()
# 
# nii_img.shape

(176, 256, 256)

In [ ]:


# plt.imshow(nii_img[79,:,:],cmap='gray')
# plt.imshow(nii_img[70,:,:],cmap='gray') #79
# plt.imshow(nii_img[88,:,:],cmap='gray')
# plt.imshow(nii_img[79,:,:],cmap='gray')
# plt.imshow(nii_img[67,:,:],cmap='gray')
# plt.imshow(nii_img[67,:,:],cmap='gray')
# plt.imshow(np.mean(nii_img[70:79,:,:],axis=0),cmap='gray')


# plt.imshow(np.mean(nii_img[80:86,:,:],axis=0),cmap='gray')
# plt.axis('off')
# plt.savefig('sagittal.png')


In [ ]:
# plt.imshow(nii_img[:,101,:],cmap='gray')
plt.imshow(np.mean(nii_img[:,85:91,:],axis=1),cmap='gray')
# plt.imshow(nii_img[:,90,:],cmap='gray') #96
# plt.imshow(nii_img[:,103,:],cmap='gray')
# plt.imshow(np.mean(nii_img[:,103:110,:],axis=1),cmap='gray')
plt.axis('off')
plt.savefig('coronal.png')

In [ ]:
# plt.imshow(nii_img[:,:,101],cmap='gray')
# plt.imshow(np.mean(nii_img[:,:,103:110],axis=-1),cmap='gray')
plt.imshow(np.mean(nii_img[:,:,116:122],axis=-1),cmap='gray')
# plt.imshow(nii_img[:,:,102],cmap='gray') #115
# plt.imshow(nii_img[:,:,90],cmap='gray') 
# plt.imshow(nii_img[:,:,96],cmap='gray')
# plt.imshow(nii_img[:,:,103:110],cmap='gray') #102, 108,112,110
plt.axis('off')
plt.savefig('axial.png')

In [ ]:
nii_img2= nibabel.load('/content/smwc1sub-OAS31002_ses-d4948_run-02_T1w.nii').get_fdata()

In [ ]:
# plt.imshow(nii_img2[54,:,:],cmap='gray') 
# plt.imshow(nii_img2[36,:,:],cmap='gray') 
plt.imshow(np.mean(nii_img2[30:36,:,:],axis=0),cmap='gray') 
# plt.imshow(np.mean(nii_img[70:79,:,:],axis=0),cmap='gray')
plt.axis('off')
plt.savefig('sagittal_preprocessed.png')

In [ ]:
# plt.imshow(nii_img2[:,54,:],cmap='gray')
plt.imshow(np.mean(nii_img2[:,42:48,:],axis=1),cmap='gray') 
plt.axis('off')
plt.savefig('coronal_preprocessed.png')

In [ ]:
# plt.imshow(nii_img2[:,:,36],cmap='gray') #54
plt.imshow(np.mean(nii_img2[:,:,36:42],axis=-1),cmap='gray') 
# plt.imshow(nii_img2[:,:,36],cmap='gray')
# plt.imshow(np.mean(nii_img2[:,:,36:42],axis=-1),cmap='gray')  
plt.axis('off')
plt.savefig('axial_preprocessed.png')

In [ ]:
np.max(img['data']),np.max(img['data']),np.min(img['data']),np.min(img['data'])

(1.0, 1.0, 0.0, 0.0)

In [ ]:
# # -*- coding: utf-8 -*-
# %cd fastcam





# import warnings
# import os
# import argparse

# import tensorflow as tf
# import math
# import numpy as np
# import torch
# import torchvision
# import torch.utils.data
# from torch.utils.data import DataLoader,TensorDataset
# import logging
# from keras.layers import Input
# from keras.layers.merge import concatenate
# import torch.nn as nn
# from torch.nn import BatchNorm3d,Conv3d,ReLU,MaxPool3d,Linear,AdaptiveAvgPool3d,Flatten,Softmax
# import torch.nn.functional as F

# from datetime import datetime
# from torch.utils import data
# import time


# import skimage.io as sio
# import os
# import shutil
# import pandas as pd
# from random import shuffle

# from skimage.transform import resize
# import skimage.io as sio
# from scipy.io import savemat,loadmat
# import cv2

# import mask
# import draw
# import norm
# import misc

# from torchvision import models

# from random import shuffle
# from torchvision.utils import make_grid, save_image

# import pandas as pd
# from gradcam.utils import visualize_cam
# from gradcam import GradCAMpp, GradCAM
# from matplotlib import pyplot as plt
# """#**Read Nii images code**"""

# import os


# def get_smoe_map(x,relu=True):
#   print(f' smoe input shape={x.shape}')
#   if relu:
#     x=tf.nn.relu(x).numpy()
#   print(f'x range={np.amax(x),np.amin(x)}')
  
#   m   = np.mean(x,axis=-1)+0.0000001 #avoid log 0
  
#   x   = x + 0.0000001
#   k   = np.log2(m) - np.mean(np.log2(x), axis=-1)
  
#   # k   = np.log10(m) - np.mean(np.log10(x), axis=-1)
#   print(np.array_equal(np.zeros(k.shape),k))
#   print(f'smoe map={k}')
#   print(f'mean={m}')
#   th  = k * m
#   print(f'smoe output shape={th.shape}')
#   return th

# def get_std_map(x):
#   print(f'before std map shape ={x.shape}')
#   m = np.std(x,axis=-1)

#   print(f'std map shape ={m.shape}')

#   return m

# def get_norm(x,const_mean=None,const_std=None):
#   # print(f'x shape={x.shape}')
#   s0      = x.shape[0]
#   s1      = x.shape[1]
#   s2      = x.shape[2]
#   # s3      = x.shape[3]
#   # print(s0,s1,s2)
#   x       = np.reshape(x,(s0,s1*s2))
#   # x       = np.reshape(x,(s0,s1*s2*s3))
#   # print(x)
#   '''
#       Compute Mean
#   '''
#   if const_mean is None:
#       m       = np.mean(x,axis=-1)
#       m       = np.reshape(m,(m.shape[0],1))
#   else:
#       m       = const_mean
      
#   '''
#       Compute Standard Deviation
#   '''
#   if const_std is None:
#       s       = np.std(x,axis=-1)
#       s       = np.reshape(s,(s.shape[0],1))
#   else:
#       s       = const_std
  
#   '''
#       The normal cumulative distribution function is used to squash the values from within the range of 0 to 1
#   '''
#   # print(s)
#   s=torch.tensor(s)
#   x       = 0.5*(1.0 + torch.erf((x-m)/(s*torch.sqrt(torch.tensor(2.0)))))
          
#   x       = x.reshape(s0,s1,s2)
#   # x       = x.reshape(s0,s1,s2,s3)
  
#   return x
#   # return torch.mean(x,dim=-1)

# def combine_sal_maps(smaps,output_size,weights,map_num,resize_mode='bilinear',do_relu=False):
#   bn  = smaps[0].shape[0]
#   cm  = torch.zeros((bn, 1, output_size[0], output_size[1]), dtype=smaps[0].dtype, device=smaps[0].device)
#   ww  = []
  
#   '''
#       Now get each saliency map and resize it. Then store it and also create a combined saliency map.
#   '''
#   for i in range(len(smaps)):
#       # assert torch.is_tensor(smaps[i]), "Each saliency map must be a Torch Tensor."
#       wsz = smaps[i].shape
#       w   = np.reshape(smaps[i],(wsz[0], 1, wsz[1], wsz[2]))#smaps[i].reshape(wsz[0], 1, wsz[1], wsz[2])
#       # w=torch.from_numpy(w)
#       # print(type(w))
#       w   = nn.functional.interpolate(w, size=output_size, mode=resize_mode, align_corners=False) 
#       ww.append(w)        # should we weight the raw maps ... hmmm
      
#       cm  += (w * weights[i])

#   '''
#       Finish the combined saliency map to make it a weighted average.
#   '''
#   weight_sum =sum(weights)
#   cm  = cm / weight_sum
#   cm  = cm.reshape(bn, output_size[0],output_size[1])
  
#   ww  = torch.stack(ww,dim=1)
#   ww  = ww.reshape(bn, map_num, output_size[0], output_size[1])
  
#   # if do_relu:
#   #     cm = F.relu(cm)
#   #     ww = F.relu(ww)
  
#   return cm, ww

# def compute_saliency_tf(base_path,inputs,tf_model,layer_end):

#   gender=inputs[1]
#   gender=tf.reshape(gender,[1,1])
#   img=inputs[0]
#   img_chunk=tf.convert_to_tensor(img)
#   print(img_chunk.shape)
#   img_chunk = tf.reshape(img_chunk,[1,121,145,6])
#   layers=[layer.name for layer in tf_model.layers]
#   outputs=[]
#   for l in layers:
#     # if l in ['activation','activation_2','activation_1']:
#     if l.startswith('activation'):
#     # if l.startswith('batch_normalization') or l in ['conv3d_48','conv3d_49','conv3d_50','conv3d_63',\
#     # 
#         outputs.append(tf_model.get_layer(name=l).output) 
#   outputs.append(tf_model.output)                                         
#   test_tf_model=tf.keras.models.Model([tf_model.inputs], outputs)
#   # test_tf_model=tf.keras.models.Model([tf_model.inputs], [tf_model.get_layer(name=layers[2]).output,tf_model.get_layer(name=layers[6]).output,tf_model.get_layer(name=layers[9]).output, tf_model.output])

#   predictions = test_tf_model([img_chunk,gender])

#   # hooks=[predictions[0],predictions[1],predictions[2],predictions[8],predictions[14],predictions[20],predictions[23]\
#   #        ,predictions[29],predictions[35],predictions[41],predictions[47],predictions[50],\
#   #        predictions[56],predictions[62],predictions[65]]#predictions[:layer_end]
#   # hooks= [predictions[0],predictions[2],predictions[17],predictions[47],predictions[62]] #1x1 and 3x3 cnn
#   hooks=[predictions[0],predictions[2],predictions[14],predictions[47],predictions[65]] #1x1 cnn
#   # hooks=predictions[:9]+predictions[-9:layer_end]

#   # choose specific channels / filters
#   for x in hooks:
#     print('ouput shapes layerwise')
#     print(x.shape)
#   #   print(x.numpy().shape)
#   #   print(np.mean(x.numpy(),axis=-2)[:,:,:,-1].shape)
#   # sal_maps       = [ get_norm(get_smoe_map(np.expand_dims(np.mean(x.numpy()[:,:,:,:,:],axis=-2)[:,:,:,2],axis=-1))) for x in hooks ]

#   #smoe
#   sal_maps       = [ get_norm(get_smoe_map(np.mean(x.numpy()[:,:,:,:,:],axis=-2))) for x in hooks ]

#   #std dev
#   # sal_maps       = [ get_norm(get_std_map(np.mean(x.numpy()[:,:,:,:,:],axis=-2))) for x in hooks ]

#   # sal_maps       = [ get_norm(get_smoe_map(x.numpy()[:,:,:,:,:])) for x in hooks ]
#   for smaps in sal_maps:
#     print(smaps.shape)
#   # return
#   # sal_maps = [np.reshape(smaps.numpy(),(121,121)) for smaps in sal_maps]

#   # all layer scale maps with equal weightage
#   weights=np.ones(len(hooks))
#    # all layer scale maps with progressive increasing weightage
#   # weights=[i+1 for i in range(len(hooks))]
#   # weights = [i for i in range(len(hooks),0,-1)]
#   map_num=len(hooks)

#   f, axarr = plt.subplots(1,1,figsize=(10,10))
#   raw=np.mean(img_chunk[0,:,:,:],axis=-1)
#   raw= raw/np.max(raw)
#   r=axarr.imshow(raw,cmap='jet')
#   axarr.set_title('Input image mean along 3rd dimension')
#   plt.colorbar(r,fraction=0.01, pad=0.04)
#   plt.savefig(base_path+'mean_input_chunk.png')

#   csal_maps,sal_maps = combine_sal_maps(sal_maps,output_size=[in_height,in_width],weights=weights,map_num=map_num)
#   output_path = base_path +'Map_Combined.png'
#   f, axarr = plt.subplots(1,1,figsize=(10,10))
#   csal_map=csal_maps[0,:,:].numpy()
#   imcs=csal_map/np.max(csal_map)
#   im = axarr.imshow(imcs,cmap='jet')
#   axarr.set_title('Combined saliency map')
#   plt.colorbar(im,fraction=0.01, pad=0.04)
#   plt.savefig(output_path)

#   il = [sal_maps[0,i,:,:] for i in range(map_num)] # Put each saliency map into the figure
#   il.append(csal_maps[0,:,:])                       # add in the combined map at the end of the figure
#   images        = [torch.stack(il, 0)]          
#   images        = make_grid(images, nrow=5)
#   sal_img=images.unsqueeze(1)
#   output_path=base_path +'Sal_Maps.png'
#   save_image(sal_img,output_path)

#   input_path = output_path
#   f, axarr = plt.subplots(1,1,figsize=(10,10))
#   im=sio.imread(input_path)
#   im=axarr.imshow(np.mean(im,axis=-1)/255, cmap='jet');
#   axarr.set_title('layerwise saliency maps')
#   plt.colorbar(im,fraction=0.01, pad=0.04)
#   output_path=base_path +'Sal_Maps_jet.png'
#   plt.savefig(output_path)
#   return csal_maps



# # """#**Saliency Map section begins**"""



# # Commented out IPython magic to ensure Python compatibility.
# # %cd fastcam/
# from random import shuffle
# def get_scan_from_subjects(data_path,subject_ids,label_df):
#     scans=[]
#     labels=[]
#     gender=[]
#     cdr=[]
#     ids=[]
#     subject_ids = set(subject_ids)
#     for subject in subject_ids :
#         path=os.path.join(data_path,subject)
#         paths=os.listdir(path)
#         ids.extend([scan_id.split('.')[0] for scan_id in paths])
#         scans.extend([ os.path.join(path,scan_path) for scan_path in paths])
    
#         labels.extend([label_df[label_df['MRI ID']==scan_id.split('.')[0]]['Age'].to_list()[0] for scan_id in paths ])
#         gender.extend([label_df[label_df['MRI ID']==scan_id.split('.')[0]]['M/F'].to_list()[0] for scan_id in paths ])
#         cdr.extend([label_df[label_df['MRI ID']==scan_id.split('.')[0]]['CDR'].to_list()[0] for scan_id in paths ])
  
#     return scans,labels,gender,ids,cdr

# def get_test_files(label_path,data_path,debug_mode_subject=None):

#     data = pd.read_csv(label_path)
#     data = data.rename(columns={'MR ID':'MRI ID'})
#     data['M/F'] = encode_gender(data)
#     if debug_mode_subject is None:
#       test_ids = os.listdir(data_path)
#     else:
#       test_ids=debug_mode_subject

#     shuffle(test_ids)
#     test_patients,test_labels,test_gender,scan_ids,test_cdr = get_scan_from_subjects(data_path,test_ids,data)

#     return test_patients,scan_ids, test_labels,test_gender,test_cdr

# def encode_gender(data):
#     data['M/F'] = pd.Categorical(data['M/F'])
    
#     return data['M/F'].cat.codes

# def parse_function_image(example_proto):

#     features = {
#         'image': tf.io.FixedLenFeature([], tf.string),
#         'image_shape': tf.io.FixedLenFeature([], tf.string)
#     }

#     content = tf.io.parse_single_example(example_proto, features=features)

#     content['image_shape'] = tf.io.decode_raw(content['image_shape'], tf.int32)
#     content['image'] = tf.io.decode_raw(content['image'], tf.float32)
#     content['image'] = tf.reshape(content['image'], content['image_shape'])

#     return content['image']



# import cv2
# import numpy as np
# # LAYER_NAME='conv3d'
# def get_grads(layer_name,tf_model,inputs):

#   cam_list=[]
#   gender= inputs[1] #tf.constant([[1]],dtype=tf.float32)
#   gender=tf.reshape(gender,[1,1])
#   img=inputs[0]
#   grad_model = tf.keras.models.Model([tf_model.inputs], [tf_model.get_layer(name=layer_name).output, tf_model.output])
#   # for i in range(48,54,6):
#   # end=start+6
#   # print(f'{start,end}')
#   # img_chunk2=
#   img_chunk=tf.convert_to_tensor(img)
#   img_chunk = tf.reshape(img_chunk,[1,121,145,6])

#   cdr_ohe_dict={0:[1.0,0.0,0.0,0.0],0.5:[0.0,1.0,0.0,0.0],1:[0.0,0.0,1.0,0.0],2:[0.0,0.0,0.0,1.0]}
#   cdr_keys= list(cdr_ohe_dict.keys())
#   with tf.GradientTape() as tape:
#       conv_outputs, predictions = grad_model([img_chunk,gender])
#       print(f'predictions={predictions}')
#       # loss=predictions
#       loss = predictions[0] 
     

#   output = conv_outputs[0]#[0,:,:,:,100]
#   print(f'entering tape gradients')

#   grads = tape.gradient(loss, conv_outputs)[0]#[0,:,:,:,100]
#   print(type(grads))
#   print(f'Crossed tape gradients')
#   gate_f = tf.cast(output > 0, 'float32')
#   gate_r = tf.cast(grads > 0, 'float32')
#   # now there are 2 choice either use grads(raw grads) or use guided grads)
#   guided_grads = tf.cast(output > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

#   print(f'Entering reduce mean using guided_grads with shape={guided_grads.shape}')
#   #guided grads
#   weights = tf.reduce_mean(guided_grads, axis=(0,1,2))


#   print(f'Computing CAM using output with shape:{output.shape}')

#   print(f'weights={weights.shape}')
#   cam = np.zeros(output.shape[0:3], dtype=np.float32)
#   print(cam.shape)


#   cam=tf.reduce_sum(tf.multiply(output,weights),axis=-1)
#   cam_list.append(cam)
#   return cam_list,grads,loss,weights,output,img_chunk

# from skimage.transform import resize
# from matplotlib import pyplot as plt
# import os
# import torch
# # p='/content/drive/My Drive/Uni-Sem4/'+mname+'/layer_'+LAYER_NAME+'/'

# def compute_gcam_and_gcam_pp(layer_name,model,inputs):
#   # print(f'{inputs[0].shape,inputs[1].shape}')
#   cam_list,grads,loss,weights,output,img_chunk = get_grads(layer_name,model,inputs)
  
#   heatmap_list=[]
#   for i,cam in enumerate(cam_list):#as we are doing chunk wise so this camlist will have only one cam

#     print(f'cam shape={cam.shape}')
    
#     #gcam
#     cam_map=resize(cam,(img_chunk.shape[1],img_chunk.shape[2],img_chunk.shape[3]))

#     cam_map = np.maximum(cam_map,0)
#     original_image=img_chunk.numpy()
   
#     heatmap = (cam_map - cam_map.min()) / (cam_map.max() - cam_map.min())

  
#     print(original_image.shape)
#     image=np.mean(original_image[0,:,:,:],axis=-1)
#     print(image.shape)

#     mri_img=image#np.squeeze(image)
#     heatmap_list.append(heatmap)
#     # print(heatmap.min(),heatmap.max())

#     heatmap_gcam = (cam_map - cam_map.min()) / (cam_map.max() - cam_map.min())

      
      
#     gcam_img=(np.mean(heatmap_gcam,axis=-1)* 255).astype("uint8")
   
#     #gcam++
#     print(f'grads shape ={grads.shape},tf.exp(loss) shape={tf.exp(loss).shape}')
#     conv_first_grad = tf.exp(loss)[0]*grads
#     #second_derivative
#     conv_second_grad = tf.exp(loss)[0]*grads*grads
#     #triple_derivative
#     conv_third_grad = tf.exp(loss)[0]*grads*grads*grads
    
#     global_sum = np.sum(tf.reshape(output,(-1,conv_first_grad[0].shape[2])), axis=0)
#     print(f'conv_first_grad shape={conv_first_grad.shape},conv_second_grad shape={conv_second_grad.shape} ,  conv_third_grad shape={conv_third_grad.shape}, global_sum.shape={global_sum.shape}  ')
#     alpha_num = conv_second_grad[0]
#     # alpha_denom = conv_second_grad[0]*2.0 + conv_third_grad[0]*global_sum.reshape((1,1,conv_first_grad[0].shape[2]))
#     alpha_denom = conv_second_grad*2.0 + conv_third_grad*global_sum.reshape((1,1,1,conv_first_grad[0].shape[2]))
#     alpha_denom = np.where(alpha_denom != 0.0, alpha_denom, np.ones(alpha_denom.shape))
#     alphas = alpha_num/alpha_denom

    

#     alphas_thresholding = np.where(weights, alphas, 0.0)
#     print(f'alphas_thresholding shape={alphas_thresholding.shape}')
#     alpha_normalization_constant = np.sum(np.sum(alphas_thresholding, axis=0),axis=0)
#     alpha_normalization_constant_processed = np.where(alpha_normalization_constant != 0.0, alpha_normalization_constant, np.ones(alpha_normalization_constant.shape))
#     print(f'alpha_normalization_constant_processed shape={alpha_normalization_constant_processed.shape}')
    
#     # alphas /= alpha_normalization_constant_processed.reshape((1,1,conv_first_grad[0].shape[2]))
#     alphas /= alpha_normalization_constant_processed.reshape((1,1,3,conv_first_grad[0].shape[2]))
#     print(f'weights.shape={weights.shape},alphas.shape={alphas.shape}')
#     weights_alpha=tf.reduce_sum(tf.multiply(weights,alphas),axis=0)
    
#     cam=tf.reduce_sum(tf.multiply(output,weights_alpha),axis=-1)
    
#     cam_map=resize(cam,(img_chunk.shape[1],img_chunk.shape[2],img_chunk.shape[3]))
  
    
#     print(f'cam_map={cam_map.shape}')
#     cam_map = np.maximum(cam_map, 0)

#     heatmap_gcam_pp = (cam_map - cam_map.min()) / (cam_map.max() - cam_map.min())


#     gcam_pp_img=(np.mean(heatmap_gcam_pp,axis=-1) * 255).astype("uint8")
#     ##
#     print(img_chunk.shape,mri_img.shape,gcam_img.shape,type(mri_img),type(gcam_img))
  
        
#     return image, gcam_img,gcam_pp_img

# def combine_sal_gcam(base_path,csmap,gcam_img,gcam_pp_img,image,layer_name ):
#   print(gcam_img.shape,csmap.shape,gcam_pp_img.shape,image.shape)
#   #gcam
#   if np.max(gcam_img) ==0:
#     gcam_img = gcam_img+0.001
#   if np.max(gcam_pp_img) ==0:
#     gcam_pp_img = gcam_pp_img+0.001
#   gcam_img_tensor=torch.from_numpy(gcam_img).unsqueeze(0)
#   mask_gcam = csmap*(gcam_img_tensor)
#   mask_gcam=mask_gcam/mask_gcam.max()
#   raw_tensor=torch.from_numpy(image).unsqueeze(0)
#   heatmap_gcam, result_gcam = visualize_cam(mask_gcam, raw_tensor)
#   getMask                 = mask.SaliencyMaskDropout(keep_percent = 0.1, scale_map=False)
#   hard_masked_gcam,_       = getMask(raw_tensor.unsqueeze(0),mask_gcam)#.squeeze(0))
#   hard_masked_gcam        = hard_masked_gcam.squeeze(0)
#   masked_gcam             = misc.AlphaMask(raw_tensor, mask_gcam.squeeze(0)).squeeze(0)
#   # print(masked_gcam.shape,type(masked_gcam))
#   mx= str(np.max(masked_gcam.numpy()))
#   plt.imsave(base_path+'masked_gcam_unnormalized_{0}max.png'.format(mx),masked_gcam.numpy(),cmap='jet')
#   # masked_gcam              = misc.RangeNormalize(masked_gcam)

 
#   ##

#   #gcam++
#   gcam_pp_img_tensor=torch.from_numpy(gcam_pp_img).unsqueeze(0)
#   mask_gcam_pp = csmap*(gcam_pp_img_tensor)
#   mask_gcam_pp=mask_gcam_pp/mask_gcam_pp.max()
#   raw_tensor=torch.from_numpy(image).unsqueeze(0)
#   heatmap_gcam_pp, result_gcam_pp = visualize_cam(mask_gcam_pp, raw_tensor)

#   hard_masked_gcam_pp,_       = getMask(raw_tensor.unsqueeze(0),mask_gcam_pp)#.squeeze(0))
#   hard_masked_gcam_pp         = hard_masked_gcam_pp.squeeze(0)
#   masked_gcam_pp           = misc.AlphaMask(raw_tensor, mask_gcam_pp.squeeze(0)).squeeze(0)
#   mx= str(np.max(masked_gcam_pp.numpy()))
#   plt.imsave(base_path+'masked_gcam_pp_unnormalized_{0}max.png'.format(mx),masked_gcam_pp.numpy(),cmap='jet')
#   # masked_gcam_pp           = misc.RangeNormalize(masked_gcam_pp)

#   #input image
#   #log gcam++
 

#   #
#   output_path   = base_path+"raw_img.png"
#   savemat(output_path.split('.png')[0] +'.mat',{'data':raw_tensor.numpy() ,'shape':raw_tensor.shape})
#   # plt.imsave(output_path,gcam_img)#provision for png if required.
#   base_path+='_'+layer_name

#   #save gcam and gcam++ fig
#   vmin=np.amin([np.min(gcam_img),np.min(gcam_pp_img)])
#   vmax=np.amax([np.max(gcam_img),np.max(gcam_pp_img)])

#   f, axarr = plt.subplots(1,2,figsize=(10,10))
#   img_plot = axarr[0].imshow(gcam_img,vmin=vmin,vmax=vmax, cmap='jet');
#   axarr[0].set_title('Gradcam')
#   img_plot = axarr[1].imshow(gcam_pp_img,vmin=vmin,vmax=vmax, cmap='jet');
#   axarr[1].set_title('Gradcam++')
#   plt.colorbar(img_plot,fraction=0.046, pad=0.04)
#   plt.savefig(base_path+'gcam_gcam++_fig.png')
#   #gcam
# # 
#   vmin=np.amin([torch.min(raw_tensor),torch.min(csmap),torch.min(heatmap_gcam),torch.min(result_gcam),torch.min(masked_gcam),torch.min(hard_masked_gcam)])
#   vmax=np.amax([torch.max(raw_tensor),torch.max(csmap),torch.max(heatmap_gcam),torch.max(result_gcam),torch.max(masked_gcam),torch.max(hard_masked_gcam)])
#   vmin=0
#   vmax=1.0
#   f, axarr = plt.subplots(1,6,figsize=(20,20))
#   img_plot = axarr[0].imshow(torch.mean(raw_tensor,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
#   axarr[0].set_title('input')
#   img_plot = axarr[1].imshow(torch.mean(csmap,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
#   axarr[1].set_title('combined saliency map')
#   img_plot = axarr[2].imshow(torch.mean(heatmap_gcam,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
#   axarr[2].set_title('saliency map + gradcam')
#   img_plot = axarr[3].imshow(torch.mean(result_gcam,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
#   axarr[3].set_title('saliency map+gradcam with alpha blend')
#   img_plot = axarr[4].imshow(masked_gcam,vmin=vmin,vmax=vmax, cmap='jet');
#   axarr[4].set_title('mask')
#   img_plot = axarr[5].imshow(hard_masked_gcam[0],vmin=vmin,vmax=vmax, cmap='jet');
#   axarr[5].set_title('hard mask')
#   plt.colorbar(img_plot,fraction=0.046, pad=0.04)
#   plt.savefig(base_path+'sal+gcam_fig.png')

#   print(hard_masked_gcam.permute([2,0,1]).shape)
#   output_path   = base_path+"gcam_img.png"
#   savemat(output_path.split('.png')[0] +'.mat',{'data':gcam_img ,'shape':gcam_img.shape})
#   output_path   = base_path+"heatmap_gcam.png"
#   savemat(output_path.split('.png')[0] +'.mat',{'data':heatmap_gcam.permute([1,2,0]).numpy() ,'shape':heatmap_gcam.permute([1,2,0]).numpy().shape})
#   output_path   = base_path+"result_gcam.png"
#   savemat(output_path.split('.png')[0] +'.mat',{'data':result_gcam.permute([1,2,0]).numpy() ,'shape':result_gcam.permute([1,2,0]).numpy().shape})
#   output_path   = base_path+"hard_masked_gcam.png" 
#   savemat(output_path.split('.png')[0] +'.mat',{'data':hard_masked_gcam.permute([1,2,0]).numpy() ,'shape':hard_masked_gcam.permute([1,2,0]).shape})
#   output_path   = base_path+"masked_gcam.mat" 
#   savemat(output_path,{'data':masked_gcam.numpy() ,'shape':masked_gcam.numpy().shape})
#   masked_gcam_mat = loadmat(output_path)['data']
#   plt.clf()
#   p=plt.imshow(masked_gcam_mat,cmap='jet')
#   plt.colorbar(p)      
#   plt.clim(0.8,1)
#   output_path   = base_path+"masked_gcam_0.8.png" 
#   plt.savefig(output_path)


#   #gcam_pp
#   vmin=np.amin([torch.min(raw_tensor),torch.min(csmap),torch.min(heatmap_gcam_pp),torch.min(result_gcam_pp),torch.min(masked_gcam_pp),torch.min(hard_masked_gcam_pp)])
#   vmax=np.amax([torch.max(raw_tensor),torch.max(csmap),torch.max(heatmap_gcam_pp),torch.max(result_gcam_pp),torch.max(masked_gcam_pp),torch.max(hard_masked_gcam_pp)])

#   vmin=0
#   vmax=1.0

#   f, axarr = plt.subplots(1,6,figsize=(20,20))
#   img_plot = axarr[0].imshow(torch.mean(raw_tensor,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
#   axarr[0].set_title('input')
#   img_plot = axarr[1].imshow(torch.mean(csmap,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
#   axarr[1].set_title('combined saliency map')
#   img_plot = axarr[2].imshow(torch.mean(heatmap_gcam_pp,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
#   axarr[2].set_title('saliency map + gradcam++')
#   img_plot = axarr[3].imshow(torch.mean(result_gcam_pp,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
#   axarr[3].set_title('saliency map+gradcam++ with alpha blend')
#   img_plot = axarr[4].imshow(masked_gcam_pp,vmin=vmin,vmax=vmax, cmap='jet');
#   axarr[4].set_title('mask')
#   img_plot = axarr[5].imshow(hard_masked_gcam_pp[0],vmin=vmin,vmax=vmax, cmap='jet');
#   axarr[5].set_title('hard mask')
#   plt.colorbar(img_plot,fraction=0.046, pad=0.04)
#   plt.savefig(base_path+'sal+gcam++_fig.png')


#   raw_img = torch.mean(raw_tensor,axis=0).numpy()
#   output_path   = base_path+"raw_input.png"
#   savemat(output_path.split('.png')[0] +'.mat',{'data':raw_img ,'shape':raw_img.shape})

#   f, axarr = plt.subplots(1,1,figsize=(10,10))

#   r=axarr.imshow(raw_img,cmap='gray')
#   axarr.set_title('raw gray image')
#   cbar=plt.colorbar(r,fraction=0.046, pad=0.04)
#   cbar.set_clim(0,1)
#   plt.savefig(base_path+'raw_gray_cbar.png')

#   csmap_img = torch.mean(csmap,axis=0).numpy()
#   output_path   = base_path+"csmap.png"
#   savemat(output_path.split('.png')[0] +'.mat',{'data':csmap_img ,'shape':csmap_img.shape})
#   output_path   = base_path+"gcam_pp_img.png"
#   savemat(output_path.split('.png')[0] +'.mat',{'data':gcam_pp_img ,'shape':gcam_pp_img.shape})
#   output_path   = base_path+"heatmap_gcam_pp.png"
#   savemat(output_path.split('.png')[0] +'.mat',{'data':heatmap_gcam_pp.permute([1,2,0]).numpy() ,'shape':heatmap_gcam_pp.permute([1,2,0]).numpy().shape})
#   output_path   = base_path+"result_gcam_pp.png"
#   savemat(output_path.split('.png')[0] +'.mat',{'data':result_gcam_pp.permute([1,2,0]).numpy() ,'shape':result_gcam_pp.permute([1,2,0]).numpy().shape})
#   output_path   = base_path+"hard_masked_gcam_pp.png" 
#   savemat(output_path.split('.png')[0] +'.mat',{'data':hard_masked_gcam_pp.permute([1,2,0]).numpy() ,'shape':hard_masked_gcam_pp.permute([1,2,0]).numpy().shape})
#   output_path   = base_path+"masked_gcam_pp.mat" 
#   savemat(output_path,{'data':masked_gcam_pp.numpy() ,'shape':masked_gcam_pp.numpy().shape})
#   masked_gcam_pp_mat = loadmat(output_path)['data']
#   plt.clf()
#   p=plt.imshow(masked_gcam_pp_mat,cmap='jet',vmin=np.min(masked_gcam_pp_mat),vmax=np.max(masked_gcam_pp_mat))
#   plt.colorbar(p)      
#   plt.clim(0.8,1)
#   output_path   = base_path+"masked_gcam_pp_0.8fig.png" 
#   plt.savefig(output_path)
# l
#   ## GCAM
#   masked_gcam=masked_gcam.numpy()

#   frac=0.5

#   t='masked_gcam'
#   max =  1 #np.amax(masked_gcam)
#   r1=(masked_gcam/max)

#   r1[np.where(r1<frac*np.max(r1))]=0
#   # r1[np.where(r1<np.median(r1))]=0
#   plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet',vmin=np.min(r1),vmax=np.max(r1))

#   frac=0.3
#   r1=(masked_gcam/max)
#   print(f'Min Max values of Sal+GCAM map ={np.min(r1),np.max(r1)}')
#   r1[np.where(r1<frac*np.max(r1))]=0
#   # r1[np.where(r1<np.median(r1))]=0
  
#   plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet',vmin=np.min(r1),vmax=np.max(r1))


#   plt.clf()
#   p=plt.imshow(masked_gcam_mat,cmap='jet',vmin=np.min(r1),vmax=np.max(r1))
#   plt.colorbar(p)      
#   plt.clim(0,1)
#   output_path   = base_path+"masked_gcam_0.3_fig.png" 
#   plt.savefig(output_path)

#   frac=0.8
#   r1=(masked_gcam/max)

#   r1[np.where(r1<frac*np.max(r1))]=0
#   # r1[np.where(r1<np.median(r1))]=0
#   plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet',vmin=np.min(r1),vmax=np.max(r1))




#   ## fraction mask  map for overlaying GCAM++
#   masked_gcam_pp=masked_gcam_pp.numpy()

#   frac=0.5

#   t='masked_gcam_pp'
#   max = 1 #np.amax(masked_gcam_pp)
#   r1=(masked_gcam_pp/max)

#   r1[np.where(r1<frac*np.max(r1))]=0
#   # r1[np.where(r1<np.median(r1))]=0
#   plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet',vmin=np.min(r1),vmax=np.max(r1))

#   frac=0.3

#   r1=(masked_gcam_pp/max)
#   print(f'Min Max values of Sal+GCAM++ map ={np.min(r1),np.max(r1)}')
#   r1[np.where(r1<frac*np.max(r1))]=0
#   # r1[np.where(r1<np.median(r1))]=0
  
#   plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet',vmin=np.min(r1),vmax=np.max(r1))

#   plt.clf()
#   p=plt.imshow(masked_gcam_pp_mat,cmap='jet',vmin=np.min(r1),vmax=np.max(r1))
#   plt.colorbar(p)      
#   plt.clim(0,1)
#   output_path   = base_path+"masked_gcam_pp_0.3_fig.png" 
#   plt.savefig(output_path)


#   frac=0.8
#   r1=(masked_gcam_pp/max)

#   r1[np.where(r1<frac*np.max(r1))]=0
#   # r1[np.where(r1<np.median(r1))]=0
#   plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet',vmin=np.min(r1),vmax=np.max(r1))

#   ##





/content/fastcam
